# Data Cleaning for Avatar: The Last Airbender Dataset
    by Adam Ward

In [ ]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
from transformers import BertModel

In [15]:
# load in the data
data = pd.read_csv("ATLA-episodes-scripts.csv")
data.tail(5)

,Character,script,ep_number,Book,total_number
13364,Suki,And why did you paint me firebending?,21,3,61
13365,Sokka,I thought it looked more exciting that way. [M...,21,3,61
13366,Iroh,"[Points at painting.] Hey, my belly's not that...",21,3,61
13367,Toph,Well I think you all look perfect! [They laugh.],21,3,61
13368,NaN,"Aang walks past Appa, petting him briefly, bef...",21,3,61


In [18]:
# separate the script into characters
main_chars = ["Sokka", "Katara", "Zuko", "Iroh", "Aang", "Toph", "Azula"]
for char in data["Character"].unique():
    print(char)

nan
Sokka
Katara
Zuko
Iroh
Aang
Kanna
Village girl
Southern Water Tribe boy
Village boy
Village kids
Southern Water Tribe girl
Firebender
Zhao
Soldier
Gyatso
Aang and Sokka
Fire Sage
Oyaji
Suki
Spectators
Little girl
Fisherman
Painter
Fangirls
Koko
Little Girl
Gate guard
Cabbage merchant
Bumi
Guard
Chamberlain
Haru
Haru's mother
Tax collector
Old man
Warden
Tyro
Prisoner
Captain
Kay-fon
Senlin Village leader
Earthbender captain
Fire Nation Soldier
Engineer
Great Fire Sage
Shyu
Roku
Oh
Pirate captain
Pirate
Together
Eye-patch soldier
Jet
Fire Nation soldier
Pipsqueak
The Duke
Smellerbee
Old Fire Nation civilian
Aang:
Gan Jin tribesman
Zhang leader
Canyon guide
Gan Jin leader
Gan Jin Leader
Gan Jin man
Gyatso and Katara
Merchant woman
Shopkeeper
Fisherman's wife
Lieutenant Jee
Jee
Air Nomad boy #1
Air Nomad boy #2
Tashi
Pasang
Young Zuko
Bujing
Elder soldier
Ozai
Shinu
Scout #1
Scout #2
Herbalist
Scribe
Officer
Calm man
Strange Man
Meng
Woman
Wu
Village Woman
Farmer
Poi and Ping
Man
Man 

In [16]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    
    return temp

# clean the data and put it in the format for the model
data['Text'] = data['Text'].map(lambda x : clean_tweet(x))

KeyError: 'Text'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'positive': 0,
          'negative': 1,
          'uncertainty': 2,
          'litigious': 3,
          }

In [17]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['Label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['Text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

## Predict

In [ ]:
def predict(model, new_data, model_path, batch_size=2, device='cuda' if torch.cuda.is_available() else 'cpu'):

    # Load the pretrained model's weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()  # Set model to evaluation mode

    # Prepare the new dataset
    new_dataset = Dataset(new_data)
    new_dataloader = DataLoader(new_dataset, batch_size=batch_size, shuffle=False)

    predictions = []

    with torch.no_grad():
        for data_input in tqdm(new_dataloader):
            mask = data_input['attention_mask'].to(device)
            input_id = data_input['input_ids'].squeeze(1).to(device)

            # Forward pass to get predictions
            output = model(input_id, mask)

            # Get the predicted labels (assuming it's a classification model)
            predicted_labels = output.argmax(dim=1)

            # Store the predictions
            predictions.extend(predicted_labels.cpu().numpy())

    return predictions
